### Rival posession overall

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
#df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')
df = pd.read_csv('/Users/luisalawson/Downloads/Sample Data/epl.csv', delimiter=';')

In [40]:
def match_outcome(df):
    relevant_df = df[df['description'].isin(['Goal', 'Pass'])]
    matches = relevant_df['match_id'].unique()
    match_data = []

    for match in matches:

        match_df = relevant_df[relevant_df['match_id'] == match].reset_index(drop=True)
        
        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        home_score = 0
        away_score = 0

        for i, row in match_df.iterrows():

            team_id = row['team_id']
            
            if team_id == home_team_id and row['description'] == 'Goal' and row['outcome'] == 1:
              
                if i + 1 < len(match_df):
                    next_row = match_df.iloc[i + 1]
                    
                    if next_row['team_id'] == home_team_id:
                        away_score += 1
                    else:
                        home_score += 1
                else:
                    home_score += 1
            elif team_id == away_team_id and row['description'] == 'Goal' and row['outcome'] == 1:
                
                if i + 1 < len(match_df):
                    next_row = match_df.iloc[i + 1]
                    if next_row['team_id'] == away_team_id:
                        home_score += 1
                    else:
                        away_score += 1
                else:
                    away_score += 1

        match_stats = {
            'match_id': match,
            'home_team': home_team_name,
            'away_team': away_team_name,
            'home_score': home_score,
            'away_score': away_score
        }

        match_data.append(match_stats)

    return pd.DataFrame(match_data)

In [41]:
matches_outcome =match_outcome(df)

In [42]:
excel_file_path = '/Users/luisalawson/Downloads/prueba.xlsx'  

matches_outcome.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/prueba.xlsx


In [50]:
df['x_float'] = df['x'].str.replace(',', '.').astype(float)
dangerous_area = (70, 100)
df['dangerous_zone'] = df['x_float'].apply(lambda x: 1 if dangerous_area[0] < x < dangerous_area[1] else 0)

In [74]:
def group_plays(df):
    match_stats_list = []
    unique_matches = df['match_id'].unique()

    for currentmatch in unique_matches:
        match_df = df[df['match_id'] == currentmatch]

        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        # print(f"Processing match: {currentmatch}")
        # print(f"Home Team: {home_team_name} (ID: {home_team_id})")
        # print(f"Away Team: {away_team_name} (ID: {away_team_id})")
        
        home_plays = 0
        away_plays = 0
        home_passes = 0
        away_passes = 0
        home_dangerous_play = 0
        away_dangerous_play = 0
        
        current_team = None
        last_event_id = None
        in_play = False
        current_passes = 0
        play_in_danger_zone = []


        for i, row in match_df.iloc[4:].iterrows():
            team_id = row['team_id']
            event_id = row['event_id']
            dangerous_area = row['dangerous_zone']
            play_in_danger_zone.append(dangerous_area)
           
            
            # print(f"\nEvent {i}:")
            # print(f"Team ID: {team_id}, Event ID: {event_id}")
            # print(f"Event in dangerous zone: {row['dangerous_zone']}")

            if not in_play:
                in_play = True
                current_team = team_id
                last_event_id = event_id
                current_passes = 0

                # print("New play started")
            else:

                if team_id != current_team or event_id != (last_event_id + 1):
                    # print(f"Play ended for Team ID: {current_team}")
                    if current_team == home_team_id:
                        home_plays += 1
                        home_passes += current_passes
                        # print(f"Home plays: {home_plays}, Home passes: {home_passes}")
                        # print(play_in_danger_zone)
                        if sum(play_in_danger_zone)>0:
                            home_dangerous_play += 1
                        #     print("Home dangerous play recorded")
                        # print(f"Home plays: {home_plays}, Home passes: {home_passes}")

                    else:
                        away_plays += 1
                        away_passes += current_passes
                        # print(f"Away plays: {away_plays}, Away passes: {away_passes}")
                        # print(play_in_danger_zone)
                        if sum(play_in_danger_zone)>0:
                            away_dangerous_play += 1
                        #     print("Away dangerous play recorded")
                        # print(f"Away plays: {away_plays}, Away passes: {away_passes}")

                    current_team = team_id
                    last_event_id = event_id
                    current_passes = 0
                    play_in_danger_zone = []
                    # print("New play started")
                else:

                    last_event_id = event_id
                    # print("Event continued within the same play")
        
            if row['description'] == 'Pass' and row['outcome'] == 1:
                current_passes += 1
                # print(f"Pass completed by Team ID: {team_id}. Current passes in play: {current_passes}")
        
        # print(f"\nFinal stats for match {currentmatch}:")
        # print(f"Home plays: {home_plays}, Home passes: {home_passes}, Home dangerous plays: {home_dangerous_play}")
        # print(f"Away plays: {away_plays}, Away passes: {away_passes}, Away dangerous plays: {away_dangerous_play}")

        match_stats = {
            'match_id': currentmatch,
            'home_team': home_team_name,
            'away_team': away_team_name,
            'home_team_plays': home_plays,
            'away_team_plays': away_plays,
            'home_passes_per_play': round(home_passes / home_plays, 4) if home_plays > 0 else 0,
            'away_passes_per_play': round(away_passes / away_plays, 4) if away_plays > 0 else 0,
            'home_dangerous_plays': home_dangerous_play,
            'away_dangerous_plays': away_dangerous_play,
            'home_RPDA': round(away_dangerous_play / away_plays, 4) if away_plays > 0 else 0,
            'away_RPDA': round(home_dangerous_play / home_plays, 4) if home_plays > 0 else 0
        }

        match_stats_list.append(match_stats)
    
    return pd.DataFrame(match_stats_list)


In [75]:
df_grouped_plays = group_plays(df)

In [76]:
df_grouped_plays

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_dangerous_plays,away_dangerous_plays,home_RPDA,away_RPDA
0,441999,Arsenal,Sunderland,313,320,2.1342,0.7438,110,117,0.3656,0.3514
1,442000,Everton,Manchester United,348,338,0.6552,1.7899,119,126,0.3728,0.3420
2,442001,Fulham,Norwich City,285,306,1.8982,1.1144,91,104,0.3399,0.3193
3,442002,Manchester City,Southampton,299,276,2.2174,1.2355,119,107,0.3877,0.3980
4,442003,Newcastle United,Tottenham Hotspur,319,311,1.1473,1.0868,95,90,0.2894,0.2978
...,...,...,...,...,...,...,...,...,...,...,...
375,442374,Swansea City,Fulham,296,296,2.1453,1.0811,107,101,0.3412,0.3615
376,442375,Tottenham Hotspur,Sunderland,378,387,1.1270,0.4470,118,155,0.4005,0.3122
377,442376,West Bromwich Albion,Manchester United,279,271,0.9857,2.0517,114,122,0.4502,0.4086
378,442377,West Ham United,Reading,331,341,0.9486,0.9267,118,105,0.3079,0.3565


In [77]:
df_dc = pd.read_csv('/Users/luisalawson/Downloads/dc_results.csv')

In [78]:
df_dc['ranking_overall'] = df_dc['dc_overall'].rank(ascending=False)
df_dc['ranking_home'] = df_dc['dc_home'].rank(ascending=False)
df_dc['ranking_away'] = df_dc['dc_away'].rank(ascending=False)

In [79]:
def get_team_rankings(row, df_dc):
    
    home_team = row['home_team']
    away_team = row['away_team']

    home_team_ranking_overall = df_dc[df_dc['team_name'] == home_team]['ranking_overall'].values
    home_team_ranking_home = df_dc[df_dc['team_name'] == home_team]['ranking_home'].values
    home_team_ranking_away = df_dc[df_dc['team_name'] == home_team]['ranking_away'].values
    home_team_dc_value_home = df_dc[df_dc['team_name'] == home_team]['dc_home'].values
    home_team_dc_value_away= df_dc[df_dc['team_name'] == home_team]['dc_away'].values

    away_team_ranking_overall = df_dc[df_dc['team_name'] == away_team]['ranking_overall'].values
    away_team_ranking_home = df_dc[df_dc['team_name'] == away_team]['ranking_home'].values
    away_team_ranking_away = df_dc[df_dc['team_name'] == away_team]['ranking_away'].values
    away_team_dc_value_home = df_dc[df_dc['team_name'] == away_team]['dc_home'].values
    away_team_dc_value_away= df_dc[df_dc['team_name'] == away_team]['dc_away'].values


    return pd.Series({
        'home_team_ranking_overall': home_team_ranking_overall[0] if len(home_team_ranking_overall) > 0 else None,
        'home_team_ranking_home': home_team_ranking_home[0] if len(home_team_ranking_home) > 0 else None,
        'home_team_ranking_away': home_team_ranking_away[0] if len(home_team_ranking_away) > 0 else None,
        'home_team_dc_value_home': home_team_dc_value_home[0]if len(home_team_dc_value_home) > 0 else None,
        'home_team_dc_value_away': home_team_dc_value_away[0]if len(home_team_dc_value_away) > 0 else None,
        'away_team_ranking_overall': away_team_ranking_overall[0] if len(away_team_ranking_overall) > 0 else None,
        'away_team_ranking_home': away_team_ranking_home[0] if len(away_team_ranking_home) > 0 else None,
        'away_team_ranking_away': away_team_ranking_away[0] if len(away_team_ranking_away) > 0 else None,
        'away_team_dc_value_home': away_team_dc_value_home[0]if len(away_team_dc_value_home) > 0 else None,
        'away_team_dc_value_away': away_team_dc_value_away[0]if len(away_team_dc_value_away) > 0 else None
    })

In [80]:
df_grouped_plays[['home_team_ranking_overall', 'home_team_ranking_home', 'home_team_ranking_away', 'home_team_dc_value_home', 'home_team_dc_value_away', 
                   'away_team_ranking_overall', 'away_team_ranking_home', 'away_team_ranking_away', 'away_team_dc_value_home','away_team_dc_value_away']] = df_grouped_plays.apply(get_team_rankings, axis=1, df_dc=df_dc)

In [81]:
df_grouped_plays

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_dangerous_plays,away_dangerous_plays,home_RPDA,...,home_team_ranking_overall,home_team_ranking_home,home_team_ranking_away,home_team_dc_value_home,home_team_dc_value_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,away_team_dc_value_home,away_team_dc_value_away
0,441999,Arsenal,Sunderland,313,320,2.1342,0.7438,110,117,0.3656,...,4.0,3.0,5.0,0.728324,0.707892,18.0,18.0,20.0,0.670015,0.646505
1,442000,Everton,Manchester United,348,338,0.6552,1.7899,119,126,0.3728,...,13.0,13.0,14.0,0.694702,0.676357,5.0,6.0,1.0,0.715204,0.717443
2,442001,Fulham,Norwich City,285,306,1.8982,1.1144,91,104,0.3399,...,6.0,5.0,4.0,0.719208,0.709649,19.0,19.0,17.0,0.651262,0.653968
3,442002,Manchester City,Southampton,299,276,2.2174,1.2355,119,107,0.3877,...,10.0,10.0,10.0,0.701668,0.681605,3.0,4.0,2.0,0.722656,0.714054
4,442003,Newcastle United,Tottenham Hotspur,319,311,1.1473,1.0868,95,90,0.2894,...,17.0,17.0,19.0,0.670972,0.646586,12.0,11.0,15.0,0.700559,0.673469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,442374,Swansea City,Fulham,296,296,2.1453,1.0811,107,101,0.3412,...,15.0,15.0,12.0,0.683717,0.677075,6.0,5.0,4.0,0.719208,0.709649
376,442375,Tottenham Hotspur,Sunderland,378,387,1.1270,0.4470,118,155,0.4005,...,12.0,11.0,15.0,0.700559,0.673469,18.0,18.0,20.0,0.670015,0.646505
377,442376,West Bromwich Albion,Manchester United,279,271,0.9857,2.0517,114,122,0.4502,...,11.0,12.0,11.0,0.696600,0.681150,5.0,6.0,1.0,0.715204,0.717443
378,442377,West Ham United,Reading,331,341,0.9486,0.9267,118,105,0.3079,...,14.0,14.0,13.0,0.689212,0.676688,20.0,20.0,18.0,0.648819,0.649343


In [82]:
condition1 = (df_grouped_plays['home_passes_per_play'] > df_grouped_plays['away_passes_per_play'])
condition2 = (df_grouped_plays['home_team_ranking_overall'] < df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] > df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition1 & condition2, 1, 0)

condition3 = (df_grouped_plays['home_passes_per_play'] < df_grouped_plays['away_passes_per_play'])
condition4 = (df_grouped_plays['home_team_ranking_overall'] > df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] < df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition3 & condition4, 1, df_grouped_plays['correct'])

In [85]:
matches_outcome

,match_id,home_team,away_team,home_score,away_score
0,441999,Arsenal,Sunderland,0,0
1,442000,Everton,Manchester United,1,0
2,442001,Fulham,Norwich City,5,0
3,442002,Manchester City,Southampton,3,2
4,442003,Newcastle United,Tottenham Hotspur,2,1
...,...,...,...,...,...
375,442374,Swansea City,Fulham,0,3
376,442375,Tottenham Hotspur,Sunderland,1,0
377,442376,West Bromwich Albion,Manchester United,5,5
378,442377,West Ham United,Reading,4,2


In [86]:
complete_df = pd.merge(df_grouped_plays, matches_outcome.drop(columns=['match_id', 'home_team', 'away_team']), left_index=True, right_index=True, how='left')

In [87]:
complete_df

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_dangerous_plays,away_dangerous_plays,home_RPDA,...,home_team_dc_value_home,home_team_dc_value_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,away_team_dc_value_home,away_team_dc_value_away,correct,home_score,away_score
0,441999,Arsenal,Sunderland,313,320,2.1342,0.7438,110,117,0.3656,...,0.728324,0.707892,18.0,18.0,20.0,0.670015,0.646505,1,0,0
1,442000,Everton,Manchester United,348,338,0.6552,1.7899,119,126,0.3728,...,0.694702,0.676357,5.0,6.0,1.0,0.715204,0.717443,1,1,0
2,442001,Fulham,Norwich City,285,306,1.8982,1.1144,91,104,0.3399,...,0.719208,0.709649,19.0,19.0,17.0,0.651262,0.653968,1,5,0
3,442002,Manchester City,Southampton,299,276,2.2174,1.2355,119,107,0.3877,...,0.701668,0.681605,3.0,4.0,2.0,0.722656,0.714054,0,3,2
4,442003,Newcastle United,Tottenham Hotspur,319,311,1.1473,1.0868,95,90,0.2894,...,0.670972,0.646586,12.0,11.0,15.0,0.700559,0.673469,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,442374,Swansea City,Fulham,296,296,2.1453,1.0811,107,101,0.3412,...,0.683717,0.677075,6.0,5.0,4.0,0.719208,0.709649,0,0,3
376,442375,Tottenham Hotspur,Sunderland,378,387,1.1270,0.4470,118,155,0.4005,...,0.700559,0.673469,18.0,18.0,20.0,0.670015,0.646505,1,1,0
377,442376,West Bromwich Albion,Manchester United,279,271,0.9857,2.0517,114,122,0.4502,...,0.696600,0.681150,5.0,6.0,1.0,0.715204,0.717443,1,5,5
378,442377,West Ham United,Reading,331,341,0.9486,0.9267,118,105,0.3079,...,0.689212,0.676688,20.0,20.0,18.0,0.648819,0.649343,1,4,2


In [88]:
excel_file_path = '/Users/luisalawson/Downloads/complete_df.xlsx'  

complete_df.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/complete_df.xlsx


### Rival posession in dangerous area

In [27]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')

# Filtrar por el partido específico y el equipo contrario (Sunderland vs Arsenal)
match_id = 441999
arsenal_team_id = 3  # Supongamos que el id del Arsenal es 3
df_match = df[df['match_id'] == match_id]

# Filtrar las jugadas del equipo rival (Sunderland)
rival_team_id = df_match[df_match['team_id'] != arsenal_team_id]['team_id'].unique()[0]
df_rival = df_match[df_match['team_id'] == rival_team_id]

# Ordenar las jugadas por tiempo para asegurar la secuencia correcta
df_rival = df_rival.sort_values(by=['min', 'sec'])

# Inicializar variables
df_rival['possession_id'] = 0
current_possession = 1

# Identificar el inicio y fin de posesiones del equipo rival
for i, row in df_rival.iterrows():
    if row['description'] in ['Interception', 'Ball recovery', 'Tackle', 'Goalkeeper Save', 'Out', 'Foul', 'Clearance', 'Pass'] and row['outcome'] == 0:
        current_possession += 1
    df_rival.at[i, 'possession_id'] = current_possession

# Identificar las posesiones que pasaron por áreas peligrosas
df_rival['in_dangerous_area'] = df_rival['x'].apply(lambda x: 70 <= x <= 100)

# Agrupar por posesión para identificar posesiones peligrosas
dangerous_plays = df_rival.groupby('possession_id')['in_dangerous_area'].max().reset_index()

# Calcular RPDA/RP para el rival de Arsenal
total_possessions = df_rival['possession_id'].nunique()
dangerous_possessions = dangerous_plays[dangerous_plays['in_dangerous_area'] == True]['possession_id'].nunique()

# Calcular el ratio RPDA/RP
rpda_rp_ratio = dangerous_possessions / total_possessions if total_possessions > 0 else 0

print(f"RPDA/RP ratio para el rival de Arsenal: {rpda_rp_ratio}")


TypeError: cannot convert the series to <class 'float'>

In [21]:
#quiero ver df_match en excel, commo lo exporto?
excel_file_path = '/Users/pgule/Documents/Sample Data/RPDA_Prueba1.xlsx'

df_rival.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")


DataFrame saved to /Users/pgule/Documents/Sample Data/RPDA_Prueba1.xlsx
